In [ ]:
%%capture
import warnings
from datetime import date, timedelta
import os
!pip install seaborn
!pip install yahoo_fin
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
from tqdm import tqdm
import pandas as pd
from datetime import datetime
import holidays
import random
from google.colab import files
from statistics import mode
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import concurrent.futures
!pip install gdown
import gdown
from scipy.stats import zscore
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io

# Autenticazione con le credenziali del servizio
credentials_path = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
if not credentials_path:
    raise ValueError("GOOGLE_APPLICATION_CREDENTIALS non è impostato.")
credentials = service_account.Credentials.from_service_account_file(credentials_path)
drive_service = build('drive', 'v3', credentials=credentials)

# Funzione per scaricare il file da Google Drive
def download_file_from_google_drive(file_id, destination):
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(destination, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))
    fh.close()

# ID del file su Google Drive e percorso di destinazione
file_id = 'GOOGLE_DRIVE_FILE_ID'  # Sostituisci con l'ID del tuo file
destination = '/content/simple_kpis.csv'  # Sostituisci con il percorso di destinazione

# Scarica il file
download_file_from_google_drive(file_id, destination)

# Leggi il file CSV
file = destination
try:
    esistente = pd.read_csv(file, usecols=range(1, len(pd.read_csv(file, nrows=1).columns)))
    print("File letto con successo")
except FileNotFoundError:
    print("File non trovato")

cartella_csv = '/content/'


In [ ]:
# Solo i tickers che ho in portafoglio

prtf = ['TRMD', 'MSFT', 'STLA', 'NVDA', 'MC.PA', 'LRCX', 'META', 'AMAT', 'CFRUY', 'URI', 'AAPL', 'DHI', 'CHRD', 'LPG'
,'SMCI','MOD']

symbol_list = prtf

all_tickers_filtered = prtf

all_tickers = prtf

print(len(prtf))

In [ ]:
from tqdm import tqdm
import pandas as pd
import yfinance as yf

def fetch_fundamental_data(symbol):
    try:
        stock = yf.Ticker(symbol).info
        return stock
    except Exception as e:
        print(f"Ignorato errore per il simbolo {symbol}: {e}")
        return None

# Inizializza una lista per conservare i dati fondamentali
fundamental_data = []

# Utilizza tqdm per aggiungere una progress bar al ciclo
for symbol in tqdm(all_tickers, desc="Fetching data"):
    data = fetch_fundamental_data(symbol)
    if data is not None:
        fundamental_data.append(data)

# Crea il DataFrame finale dai dati fondamentali raccolti
df_fundamental = pd.DataFrame(fundamental_data)

In [ ]:
df_fundamental

In [ ]:
# Definisci la funzione che esegue la moltiplicazione
def multiply_columns(df):
    df['mostRecentQuarter'] = pd.to_datetime(df['mostRecentQuarter'], unit='s').dt.date
    return df.drop(columns=['currentPrice', 'fiftyTwoWeekHigh'])

# Filtra e seleziona il DataFrame, applica la funzione e ordina
df_filtered = (
    df_fundamental
    [['shortName',  'industry','dividendYield', 'ebitdaMargins', 'recommendationKey','totalCashPerShare',
      'returnOnEquity', 'currentPrice', 'fiftyTwoWeekHigh','earningsQuarterlyGrowth','mostRecentQuarter']]
    .pipe(multiply_columns)
    .sort_values(by='dividendYield', ascending=False)
)

df_filtered
nuovo = df_filtered
nuovo

In [ ]:
esistente['mostRecentQuarter'] = pd.to_datetime(esistente['mostRecentQuarter'])
nuovo['mostRecentQuarter'] = pd.to_datetime(nuovo['mostRecentQuarter'])

esistente['shortName'] = esistente['shortName'].astype(str)
nuovo['shortName'] = nuovo['shortName'].astype(str)

esistente['industry'] = esistente['industry'].astype(str)
nuovo['industry'] = nuovo['industry'].astype(str)


In [ ]:
import pandas as pd
appeso = pd.concat([esistente,nuovo]).drop_duplicates(subset=["shortName","mostRecentQuarter"]).sort_values(by=['shortName', 'mostRecentQuarter'])


In [ ]:
appeso.to_csv('/content/simple_kpis.csv')
